In [10]:
!pip install tensorflow-model-optimization numpy tensorflow tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.1 MB/s eta 0:00:00


# Train base model on CIFAR-10

In [11]:
import tensorflow as tf

(train_imgs, train_lbls), (val_imgs, val_lbls) = tf.keras.datasets.cifar10.load_data()

train_imgs, val_imgs = train_imgs/255.0, val_imgs/255.0

test_imgs, test_lbls = val_imgs[-2000:], val_lbls[-2000:]
val_imgs, val_lbls = val_imgs[:8000], val_lbls[:8000]

In [12]:
import tensorflow_model_optimization as tfmot
import tf_keras as keras

def separable_conv(i, ch):
  x = keras.layers.DepthwiseConv2D((3,3), padding='same')(i)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.Conv2D(ch, (1,1), padding='same')(x)
  x = keras.layers.BatchNormalization()(x)
  return keras.layers.Activation('relu')(x)

In [13]:
# def dwsepcnn_block(ch):
#   return keras.Sequential([
#     keras.layers.DepthwiseConv2D((3,3), padding='same'),
#     keras.layers.BatchNormalization(),
#     keras.layers.Activation('relu'),
#     keras.layers.Conv2D(ch, (1,1), padding='same'),
#     keras.layers.BatchNormalization(),
#     keras.layers.Activation('relu')
#   ])

Convolution base

In [14]:
input = keras.layers.Input((32,32,3))
x = keras.layers.Conv2D(16, (3, 3), padding='same')(input)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = separable_conv(x, 16)
x = keras.layers.MaxPooling2D((2,2))(x)
x = separable_conv(x, 48)
x = keras.layers.MaxPooling2D((2,2))(x)
x = separable_conv(x, 96)
x = separable_conv(x, 192)
x = keras.layers.MaxPooling2D((2,2))(x)

In [15]:
# model = keras.Sequential([
#     keras.layers.Input((32,32,3)),
#     keras.layers.Conv2D(16, (3, 3), padding='same'),
#     keras.layers.BatchNormalization(),
#     keras.layers.Activation('relu'),
#     dwsepcnn_block(16),
#     keras.layers.MaxPooling2D((2,2)),
#     dwsepcnn_block(48),
#     keras.layers.MaxPooling2D((2,2)),
#     dwsepcnn_block(96),
#     dwsepcnn_block(192),
#     keras.layers.MaxPooling2D((2,2))])

Classification head

In [16]:
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(10)(x)

In [17]:
# model.add(keras.layers.Flatten())
# model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.Dense(10))

In [18]:
#from keras.models import Model
model = keras.models.Model(input, x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 16)        448       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 16)        64        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 32, 32, 16)        0         
                                                                 
 depthwise_conv2d (Depthwis  (None, 32, 32, 16)        160       
 eConv2D)                                                        
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 16)        64    

In [19]:
loss_f = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss = loss_f, metrics=['accuracy'])

history = model.fit(train_imgs, train_lbls, epochs=10, batch_size=32,
                    validation_data=(val_imgs, val_lbls))

model.export('cifar10')

Epoch 1/10
1563/1563 [==============================] - 54s 33ms/step - loss: 1.5895 - accuracy: 0.4614 - val_loss: 1.2446 - val_accuracy: 0.5620
Epoch 2/10
1563/1563 [==============================] - 52s 33ms/step - loss: 1.1111 - accuracy: 0.6102 - val_loss: 1.0663 - val_accuracy: 0.6246
Epoch 3/10
1563/1563 [==============================] - 52s 33ms/step - loss: 0.9515 - accuracy: 0.6681 - val_loss: 1.0373 - val_accuracy: 0.6439
Epoch 4/10
1563/1563 [==============================] - 51s 33ms/step - loss: 0.8460 - accuracy: 0.7035 - val_loss: 0.9985 - val_accuracy: 0.6680
Epoch 5/10
1563/1563 [==============================] - 52s 33ms/step - loss: 0.7780 - accuracy: 0.7299 - val_loss: 0.8653 - val_accuracy: 0.7048
Epoch 6/10
1563/1563 [==============================] - 51s 33ms/step - loss: 0.7189 - accuracy: 0.7497 - val_loss: 0.8111 - val_accuracy: 0.7172
Epoch 7/10
1563/1563 [==============================] - 53s 34ms/step - loss: 0.6698 - accuracy: 0.7643 - val_loss: 0.8153 -

# Quantize the model
We need a sample from the training dataset to run inference on for full integer
quantization, because the zero point and scale need to be calculated for the activations.

In [20]:
cifar_ds = tf.data.Dataset.from_tensor_slices(train_imgs)
def representative_data_gen():
  for i_value in cifar_ds.batch(1).take(1000):
    i_value_f32 = tf.dtypes.cast(i_value, tf.float32)
    yield [i_value_f32]

tfl_conv = tf.lite.TFLiteConverter.from_saved_model('cifar10')
tfl_conv.representative_dataset = \
  tf.lite.RepresentativeDataset(representative_data_gen)
tfl_conv.optimizations = [tf.lite.Optimize.DEFAULT]
tfl_conv.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tfl_conv.inference_input_type = tf.int8
tfl_conv.inference_output_type = tf.int8

In [21]:
tfl_model = tfl_conv.convert()
print(len(tfl_model))

79600


Evaluate the quantized model using the validation dataset

In [22]:
tfl_interp = tf.lite.Interpreter(model_content=tfl_model)
tfl_interp.allocate_tensors()

i_details = tfl_interp.get_input_details()[0]
o_details = tfl_interp.get_output_details()[0]

# print(len(tfl_interp.get_input_details()))
# print(len(tfl_interp.get_output_details()))

i_quant = i_details['quantization_parameters']
o_quant = o_details['quantization_parameters']
i_scale = i_quant['scales'][0]
i_zero_point = i_quant['zero_points'][0]
o_scale = o_quant['scales'][0]
o_zero_point = o_quant['zero_points'][0]

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [23]:
def classify(i_data):
  input_data = i_data.reshape((1, 32, 32, 3))
  i_value_f32 = tf.dtypes.cast(input_data, tf.float32)
  i_value_f32 = i_value_f32 / i_scale + i_zero_point
  i_value_s8 = tf.cast(i_value_f32, dtype=tf.int8)

  tfl_interp.set_tensor(i_details['index'], i_value_s8)
  tfl_interp.invoke()
  o_pred = tfl_interp.get_tensor(o_details['index'])[0]

  return (o_pred - o_zero_point) * o_scale

In [24]:
num_correct_samples = 0

import numpy as np

for i_value, o_value in zip(val_imgs, val_lbls):
  o_pred_f32 = classify(i_value)
  if np.argmax(o_pred_f32) == o_value:
    num_correct_samples += 1

total_samples = len(list(val_imgs))
print('Accuracy: ', num_correct_samples/total_samples)

Accuracy:  0.7375


In [25]:
with open('cifar10.tflite', 'wb') as file:
  file.write(tfl_model)

!apt-get update && apt-get -qq install xxd
!xxd -i cifar10.tflite > model.h
!sed -i 's/unsigned char/const unsigned char/g' model.h
!sed -i 's/const/alignas(8) const/g' model.h

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.2 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,823 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,288 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,479 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,414 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://a

# Quantization aware training
https://www.tensorflow.org/model_optimization/guide/quantization/training_example

In [26]:
quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model = quantize_model(model)

# quantize_model requires a recompile
q_aware_model.compile(optimizer='adam',
                      loss=loss_f,
                      metrics=['accuracy'])

q_aware_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 quantize_layer (QuantizeLa  (None, 32, 32, 3)         3         
 yer)                                                            
                                                                 
 quant_conv2d (QuantizeWrap  (None, 32, 32, 16)        481       
 perV2)                                                          
                                                                 
 quant_batch_normalization   (None, 32, 32, 16)        65        
 (QuantizeWrapperV2)                                             
                                                                 
 quant_activation (Quantize  (None, 32, 32, 16)        3         
 WrapperV2)                                                  

All layers are now prefixed by "quant". Note that the resulting model is quantization aware but not quantized (e.g. the weights are float32 instead of int8).

In [27]:
train_images_subset = train_imgs[:1000]
train_labels_subset = train_lbls[:1000]

q_aware_model.fit(train_images_subset, train_labels_subset,
                  batch_size=64, epochs=2, validation_data=(val_imgs, val_lbls))

_, baseline_model_accuracy = model.evaluate(test_imgs, test_lbls, verbose=0)
#_, tfl_model_accuracy = tfl_model.evaluate(test_imgs, test_lbls, verbose=0)
_, q_aware_model_accuracy = q_aware_model.evaluate(test_imgs, test_lbls, verbose=0)


Epoch 1/2
16/16 [==============================] - 7s 275ms/step - loss: 0.4983 - accuracy: 0.8290 - val_loss: 0.8287 - val_accuracy: 0.7254
Epoch 2/2
16/16 [==============================] - 4s 254ms/step - loss: 0.3026 - accuracy: 0.9070 - val_loss: 0.7987 - val_accuracy: 0.7324


In [28]:
print('Baseline test accuracy: ', baseline_model_accuracy)
#print('Quant test accuracy: ', tfl_model_accuracy)
print('Quant aware test accuracy: ', q_aware_model_accuracy)

Baseline test accuracy:  0.7294999957084656
Quant aware test accuracy:  0.7170000076293945


## Quantize q-aware model

In [29]:
tfl_conv = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
tfl_conv.representative_dataset = \
  tf.lite.RepresentativeDataset(representative_data_gen)
tfl_conv.optimizations = [tf.lite.Optimize.DEFAULT]
tfl_conv.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tfl_conv.inference_input_type = tf.int8
tfl_conv.inference_output_type = tf.int8

quantized_q_aware_model = tfl_conv.convert()

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:863: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [30]:
with open('cifar10_q_aware.tflite', 'wb') as file:
  file.write(quantized_q_aware_model)

!xxd -i 'cifar10_q_aware.tflite' > q_aware_model.h
!sed -i 's/unsigned char/const unsigned char/g' q_aware_model.h
!sed -i 's/const/alignas(8) const/g' q_aware_model.h

In [32]:
import pandas as pd

def array_to_str(data):
    NUM_COLS = 12
    val_string = ''
    for i, val in enumerate(data):
        val_string += str(val)
        if (i+1) < len(data):
            val_string += ','
        if (i+1) % NUM_COLS == 0:
            val_string += '\n'
    return val_string

def gen_h_file(size, data, ilabel):
    str_out = f'int8_t g_test[] = '
    str_out += "\n{\n"
    str_out += f'{data}'
    str_out += '};\n'
    str_out += f'const int g_test_len = {size};\n'
    str_out += f'const int g_test_ilabel = {ilabel};\n'
    return str_out

imgs = list(zip(val_imgs, val_lbls))
cols = ['Image', 'Label']
df = pd.DataFrame(imgs, columns=cols)

ship_samples = df[df['Label'] == 8]

c_code = ''

for index, row in ship_samples.iterrows():
    i_value = np.asarray(row['Image'].tolist())
    o_value = np.asarray(row['Label'].tolist())
    o_pred_f32 = classify(i_value)

    if np.argmax(o_pred_f32) == o_value:
        i_value_f32 = i_value / i_scale + i_zero_point
        i_value_s8 = i_value_f32.astype(dtype=np.int8)
        i_value_s8 = i_value_s8.ravel()
        # Generate a string from the numpy array
        val_string  = array_to_str(i_value_s8)
        # Generate the C header file
        c_code  = gen_h_file(i_value_s8.size, val_string, "8")
        break

with open('input.h', 'w') as file:
    file.write(c_code)